# Network Prepare
This notebook prepares the final routing network.

1. Import the desired routing network
1. Add attributes
1. Add reconciled attributes
1. Add signals
1. Add elevation

Then the network will be turned into a directed network graph complete with an edge list representing the directed edges and another one representing turns. Some attribute values are reversed to account for direction (e.g., elevation, signals).

In [37]:
import geopandas as gpd
from pathlib import Path
import numpy as np

import pickle
import src.modeling_turns as modeling_turns
import src.add_attributes as add_attributes

Import the data from previous notebooks and merge them. Merge here so updates can be done at each step without having to repeat everything.

In [38]:
network_filepath = Path.home() / "Documents/BikewaySimData/Projects/gdot/networks"

In [39]:
#filtered data
links = gpd.read_file(network_filepath/'filtered.gpkg',layer='osm_links')
nodes = gpd.read_file(network_filepath/'filtered.gpkg',layer='osm_nodes')

In [40]:
#add osm data
links = add_attributes.add_osm_attr(links,network_filepath / 'osm_attr.pkl')



In [41]:
links.columns

Index(['osm_A', 'osm_B', 'osm_linkid', 'osmid', 'link_type', 'name', 'highway',
       'oneway', 'bearing', 'bridge', 'tunnel', 'bl', 'pbl', 'mu', 'speed_mph',
       'lanes', 'geometry'],
      dtype='object')

In [42]:
#reconciled data
reconciled = gpd.read_file(network_filepath/'reconciled.gpkg',layer='links',ignore_geometry=True)
#[col for col in reconciled.columns if col not in links.columns]

In [43]:
cols_to_keep = ['osm_linkid','speedlimit_range_mph','lanes_per_direction']
links = links.merge(reconciled[cols_to_keep],on='osm_linkid',how='left')
del reconciled

In [44]:
#rename
links.rename(columns={'osm_A':'A','osm_B':'B','osm_linkid':'linkid'},inplace=True)
nodes.rename(columns={'osm_N':'N'},inplace=True)

In [45]:
#signals added
links_w_signals = gpd.read_file(network_filepath/'signals_added.gpkg',layer='links',ignore_geometry=True)


In [48]:
nodes_w_signals = gpd.read_file(network_filepath/'signals_added.gpkg',layer='nodes',ignore_geometry=True)
nodes_w_signals

,N,signalid
0,67358015,NaN
1,67358019,NaN
2,67358022,NaN
3,67358027,NaN
4,67358031,NaN
...,...,...
133027,11718012146,NaN
133028,11718012147,NaN
133029,11718404908,NaN
133030,11718404910,NaN


In [46]:
#TODO change linkid to osm_linkid later
cols_to_keep = ['linkid','signal_A','signal_B']
links = links.merge(links_w_signals[cols_to_keep],on='linkid',how='left')
##del nodes_w_signals


In [ ]:
#elevation added
links_w_elevation = gpd.read_file(network_filepath/'elevation_added.gpkg',ignore_geometry=True)
links_w_elevation.columns
links_w_elevation.rename(columns={
    'a_s_c_e_n_t___m':'ascent_m',
    'd_e_s_c_e_n_t___m':'descent_m',
    'a_s_c_e_n_t___g_r_a_d_e':'ascent_grade',
    'd_e_s_c_e_n_t___g_r_a_d_e':'descent_grade',
}, inplace =True)
cols_to_keep = ['linkid','ascent_m','descent_m','ascent_grade','descent_grade','(0,2]_descent',
       '(2,4]_descent', '(4,6]_descent', '(6,10]_descent', '(10,15]_descent',
       '(15,inf]_descent', '(0,2]_ascent', '(2,4]_ascent', '(4,6]_ascent',
       '(6,10]_ascent', '(10,15]_ascent', '(15,inf]_ascent']
links = links.merge(links_w_elevation[cols_to_keep],on='linkid')
del links_w_elevation

In [ ]:
links.columns

Index(['osm_A', 'osm_B', 'osm_linkid', 'osmid', 'link_type', 'name', 'highway',
       'oneway', 'bearing', 'bridge', 'tunnel', 'bl', 'pbl', 'mu', 'speed_mph',
       'lanes', 'geometry', 'speedlimit_range_mph', 'lanes_per_direction',
       'linkid_x', 'signal_A', 'signal_B', 'linkid_y', 'ascent_m', 'descent_m',
       'ascent_grade', 'descent_grade', '(0,2]_descent', '(2,4]_descent',
       '(4,6]_descent', '(6,10]_descent', '(10,15]_descent',
       '(15,inf]_descent', '(0,2]_ascent', '(2,4]_ascent', '(4,6]_ascent',
       '(6,10]_ascent', '(10,15]_ascent', '(15,inf]_ascent'],
      dtype='object')

In [ ]:
fp = Path.home() / "Documents/BikewaySimData/Projects/gdot"
edges = gpd.read_file(fp/'networks/elevation_added.gpkg',layer="links")

In [ ]:
edges.columns

In [ ]:
#use geometry one last time
edges['length_ft'] = edges.length

#turn bridge and tunnel to boolean values
edges['tunnel'] = edges['tunnel'].notna()
edges['bridge'] = edges['bridge'].notna()

In [ ]:
#turn bike facil into one column
edges['bike_facility_type'] = np.nan
edges.loc[(edges['mu'] == 1) & (edges['bike_facility_type'].isna()),'bike_facility_type'] = 'shared-use path'
edges.loc[(edges['pbl'] == 1) & (edges['bike_facility_type'].isna()),'bike_facility_type'] = 'protected bike lane'
edges.loc[(edges['bl'] == 1) & (edges['bike_facility_type'].isna()),'bike_facility_type'] = 'bike lane'

In [ ]:
df_edges, pseudo_df, pseudo_G = modeling_turns.create_pseudo_dual_graph(edges,'A','B','linkid','oneway',True)

## Add desired attributes from links to df_edges

In [ ]:
#df_edges = df_edges.merge(edges[['linkid','geometry']])

In [ ]:
basic_cols = ['linkid', 'osmid', 'link_type', 'name', 'oneway','length_ft']

#anything that's an instance or would be better as a count value (but not a turn)
event_cols = ['bridge','tunnel']

#anything that's for the duration of the entire link and has categories
category_cols = ['link_type','highway','speedlimit_range_mph',
               'lanes_per_direction','bike_facility_type']

#reverse in tuple form (these need to be flipped if going the other direction)
rev_columns = [('ascent_m','descent_m'),
               ('ascent_grade','descent_grade'),
               ('(0,2]_ascent','(0,2]_descent'),
               ('(2,4]_ascent','(2,4]_descent'),
               ('(4,6]_ascent','(4,6]_descent'),
               ('(6,10]_ascent','(6,10]_descent'),
               ('(10,15]_ascent','(10,15]_descent'),
               ('(15,inf]_ascent','(15,inf]_descent')]

from itertools import chain
keep_cols = basic_cols + event_cols + category_cols + list(chain(*rev_columns))

In [ ]:
# attrs = ['linkid', 'osmid', 'link_type', 'name', 'highway',
#        'bridge', 'tunnel', 'bl', 'pbl', 'mu','speedlimit_range_mph',
#        'lanes_per_direction', 'up_grade', 'down_grade', 'length_ft',
#        'vehicle_separation','geometry']
df_edges = df_edges.merge(edges[keep_cols],on='linkid',how='left')

In [ ]:
df_edges

## Deal with grade
Need to flip sign of grade for reverse links

In [ ]:
# def combine_up_down_tuples(lst):
#     result = []
#     current_tuple = []

#     for item in lst:
#         if 'ascent' in item or 'descent' in item:
#             current_tuple.append(item)
#             if len(current_tuple) == 2:
#                 result.append(tuple(current_tuple))
#                 current_tuple = []

#     return result

# rev_columns = ['ascent_m','descent_m','ascent_grade','descent_grade',
#                '(0,2]_down', '(2,4]_down', '(4,6]_down',
#                '(6,10]_down', '(10,15]_down','(15,inf]_down',
#                '(0,2]_up', '(2,4]_up', '(4,6]_up', '(6,10]_up',
#                '(10,15]_up', '(15,inf]_up'
#                ]

# combined_tuples = combine_up_down_tuples(rev_columns)

for elev_columns in rev_columns:
    df_edges[elev_columns[0]] = np.where(df_edges['reverse_link'], df_edges[elev_columns[1]].abs(), df_edges[elev_columns[0]])
    #drop the down version?
    df_edges.drop(columns=elev_columns[1],inplace=True)

## Turns and Signals

In [ ]:
#add additional attributes needed for processing
source_links = edges[['linkid','osmid','link_type','name','highway']]
target_links = edges[['linkid','osmid','link_type','name','highway']]
source_links.columns = 'source_' + source_links.columns
target_links.columns = 'target_' + target_links.columns
pseudo_df = pseudo_df.merge(source_links,on='source_linkid',how='left')
pseudo_df = pseudo_df.merge(target_links,on='target_linkid',how='left')

## Turn Restrictions
Two types in OSM (represented as OSM relations):
- No (blank) turns
- Only this turn allowed

For chosen we don't need to consider turn restrictions

In [ ]:
# turn_restrictions = pd.read_csv(fp.parent/'osm_turn_restrictions.csv')
# pseudo_df = pseudo_df.merge(turn_restrictions,left_on=['source_osmid','target_osmid'],right_on=['from_way_id','to_way_id'],how='left')
# road_cond = (pseudo_df['source_link_type'] == 'road') & (pseudo_df['target_link_type'] == 'road')
# no_restr = pseudo_df['type'] == 'no'
# only_restr = pseudo_df['type'] == 'only'

# #add a remove column
# pseudo_df['remove'] = False

# #remove the no turns
# pseudo_df.loc[road_cond & no_restr,'remove'] = True

# #for only, find all instances road_cond + from source and set to True
# sources = set(turn_restrictions.loc[turn_restrictions['type']=='only','from_way_id'].tolist())
# pseudo_df.loc[road_cond & pseudo_df['source_osmid'].isin(sources) & pseudo_df['type'].isna(),'remove'] = True

# #Remove these turns and drop the added columns
# print((pseudo_df['remove']==True).sum(),'turns removed')
# pseudo_df = pseudo_df[pseudo_df['remove']==False]
# pseudo_df.drop(columns=['relation_id', 'restriction', 'from_way_id',
#        'to_way_id', 'type', 'remove'],inplace=True)

# Deal with signals
Perform two merges and use the source/target reverse link columns to determine which signal ID to keep.
- For the source link, use signal_B if reverse == False else signal_A
- For the target link, use signal_A if reverse == False else signal_B

In [ ]:
source = pseudo_df[['source_linkid','source_reverse_link']].merge(edges,left_on='source_linkid',right_on='linkid',how='left')
pseudo_df['source_signal'] = np.where(source['source_reverse_link'], source['signal_A'], source['signal_B'])

target = pseudo_df[['target_linkid','target_reverse_link']].merge(edges,left_on='target_linkid',right_on='linkid',how='left')
pseudo_df['target_signal'] = np.where(target['target_reverse_link']==False, target['signal_B'], target['signal_A'])

## Identifying signalized/unsignalized turns
- Only look at roads for now
- Filter to left/right turns per source linkid per direction
- Take the highest road classification and assign it as the cross street road classification

In [ ]:
import pandas as pd
highway_order = {
    'trunk': 0,
    'trunk_link': 1,
    'primary': 2,
    'primary_link': 3,
    'secondary': 4,
    'secondary_link': 5,
    'tertiary': 6,
    'tertiary_link': 7,
    'unclassified': 8,
    'residential': 9
}
highway_order = pd.Series(highway_order)
highway_order = highway_order.reset_index()
highway_order.columns = ['highway','order']

In [ ]:
#add highway ranking based on the above
pseudo_df['target_highway_order'] = pseudo_df['target_highway'].map(highway_order.set_index('highway')['order'])
pseudo_df['source_highway_order'] = pseudo_df['source_highway'].map(highway_order.set_index('highway')['order'])

In [ ]:
#remove straight and uturn
cond1 = pseudo_df['turn_type'].isin(['left','right'])
#only road to road for now
cond2 = (pseudo_df['source_link_type'] == 'road') & (pseudo_df['target_link_type'] == 'road')
cross_streets = pseudo_df[cond1 & cond2]

#use groupby to find the max target_highway order
cross_streets = cross_streets.groupby(['source_linkid','source_A','source_B'])['target_highway_order'].min()
cross_streets.name = 'cross_street'

#add to main df
pseudo_df = pd.merge(pseudo_df,cross_streets,left_on=['source_linkid','source_A','source_B'],right_index=True,how='left')

#change numbers back to normal
pseudo_df['cross_street_order'] = pseudo_df['cross_street']
pseudo_df['cross_street'] = pseudo_df['cross_street'].map(highway_order.set_index('order')['highway'])

# TODO Add OSM crossing into this logic
    - Signals
        - Wait on this until we have the route attributes code done
        - Add crossings in signalization
        - Majority of crossings are nodes not ways
        - Cycleway crossings typically dealt the same way
        - If meeting nodes are both crossings and within the traffic signal buffer, they're signalized crossings
            - Or if both connecting links are crossings/connect to the road etc
        - Way attributes
            - Footway = crossing
            - Highway = footway
        - Node attributes
            - Crossing = * (traffic signals/marked/etc)
            - Highway = crossing
        - Link attributes
            - Some links are labeled as crossings but this is not as consistent


In [ ]:
signalized = pseudo_df['source_signal'] == pseudo_df['target_signal']
left_or_straight =  pseudo_df['turn_type'].isin(['left','straight'])
both_road = (pseudo_df['source_link_type'] == 'road') & (pseudo_df['target_link_type'] == 'road')
cross_street = pseudo_df['cross_street_order'] <= 5

#signalized
pseudo_df.loc[signalized & both_road,'signalized'] = True
pseudo_df.loc[pseudo_df['signalized'].isna(),'signalized'] = False
# pseudo_df.loc[signalized & left_or_straight & both_road,'signalized_left_straight'] = True
# pseudo_df.loc[pseudo_df['signalized_left_straight'].isna(),'signalized_left_straight'] = False

pseudo_df.loc[-signalized & both_road & cross_street,'unsignalized'] = True
pseudo_df.loc[pseudo_df['unsignalized'].isna(),'unsignalized'] = False

#clean up
rem =  ['source_osmid', 'source_link_type', 'source_name',
       'source_highway', 'target_osmid', 'target_link_type', 'target_name',
       'target_highway', 'source_signal', 'target_signal',
       'target_highway_order', 'source_highway_order', 'cross_street',
       'cross_street_order']
pseudo_df.drop(columns=rem,inplace=True)

# Export for impedance calibration


In [ ]:
# df_edges = gpd.GeoDataFrame(df_edges,crs='epsg:2240')
df_edges.columns

In [ ]:
with (fp.parent / 'chosen.pkl').open('wb') as fh:
    export = (df_edges,pseudo_df,pseudo_G)
    pickle.dump(export,fh)

## Add geometry to examine results in QGIS

In [ ]:
#add geo
link_geo = dict(zip(links['linkid'],links['geometry']))
pseudo_df['src_geo'] = pseudo_df['source_linkid'].map(link_geo)
pseudo_df['trgt_geo'] = pseudo_df['target_linkid'].map(link_geo)
pseudo_df['geometry'] = pseudo_df[['src_geo','trgt_geo']].apply(lambda row: MultiLineString([row['src_geo'],row['trgt_geo']]),axis=1)

pseudo_df.drop(columns=['src_geo','trgt_geo'],inplace=True)
pseudo_df = gpd.GeoDataFrame(pseudo_df,crs=links.crs)

pseudo_df['source'] = pseudo_df['source'].astype(str)
pseudo_df['target'] = pseudo_df['target'].astype(str)

#check results (may need a smaller road network to test on)
pseudo_df.to_file(Path.home()/'Downloads/testing.gpkg',layer='cross_streets')